<left>
<table style="margin-top:0px; margin-left:0px;">
<tr>
  <td><img src="https://raw.githubusercontent.com/worm-portal/WORM-Figures/master/style/worm.png" alt="WORM" title="WORM" width=50/></td>
  <td><h1 style=font-size:30px>Acid Mine Drainage Activity</h1><h2>Answer Sheet</h2></td>
</tr>
</table>
</left>

In case you need it, here is a link to [the WORM database table](https://tinyurl.com/wormdb). It can be helpful for looking up mineral names. Descriptions of columns and their units can be found in the [WORM database documentation](https://worm-portal.asu.edu/docs/database/). 

---

### Activity: Destroyed by acid mine drainage!

<img src="https://raw.githubusercontent.com/worm-portal/WORM-Figures/refs/heads/master/3-Aqueous-Speciation/mass_transfer/bridge_AMD.png" alt="struts" title="Concrete Struts" width=50%/>
<br/><br/>
The year is 2050. County officials in Shamokin, Pennsylvania are investigating reports of eroding infrastructure. The reports indicate degradation of the concrete struts that support a bridge spanning a large stagnant pond. An investigation reveals that polluted runoff from the abandoned Henry Clay Sterling Mine has contaminated the pond in contact with the struts.

A task force is assembled to gather evidence that the acid mine drainage is causing destruction of the bridge struts. It is your task to estimate the extent of the damage so the county can budget for bridge repair.

For every 10 kg of water in the pond, there is 1 mole of concrete in contact with water.

If there is < 10% disintegration of concrete volume after equilibration with the pond, then \\$1M will be needed to patch up the concrete. Any more than that and the struts will need to be completely replaced for \\$5M. What do you recommend based on your model?

Hint: assume portlandite (cement) is a stand-in for concrete.

(Acid mine drainage data is adapted from [Cravotta III 2008](https://doi.org/10.1016/j.apgeochem.2007.10.011))

---

In [1]:
import aqequil

In [2]:
ae = aqequil.AqEquil(exclude_organics=True)

Loading Water-Organic-Rock-Microbe (WORM) thermodynamic databases...
Excluding ['organic_aq', 'organic_cr'] from column 'category_1'in wrm_data_latest.csv
wrm_data_latest.csv is now set as the active thermodynamic database.
Element database elements.csv is active.
Solid solution database solid_solutions.csv is active.
LogK database wrm_data_logk.csv is active.
Excluding ['organic_aq', 'organic_cr'] from column 'category_1'in wrm_data_logk.csv
LogK_S database wrm_data_logk_s.csv is active.
Excluding ['organic_aq', 'organic_cr'] from column 'category_1'in wrm_data_logk_s.csv
Loading thermodynamic database into pyCHNOSZ...


In [3]:
speciation = ae.speciate(input_filename="mine_AMD.csv",
                         alter_options=[["cattierite", "Suppress"]],
                         )

The input file column 'Eh' will be used to set sample redox state. If a another column is desired, set it manually using the redox_flag parameter.
Getting wrm_data_latest.csv ready. This will take a moment...
No 'charge_balance_on' column found in input file. Defaulting to no charge balancing for all samples.
Using wrm_data_latest.csv to speciate Henry Clay Sterling Mine
Finished!


In [4]:
speciation.save("AMD")

Saved as 'AMD.speciation'


In [5]:
import aqequil
speciation = aqequil.load("AMD")

Attempting to load AMD.speciation ...
Loaded 'AMD.speciation'


### Make sure amount remaining is set to 0.1 moles mineral per kg H2O.

This is because for every 10 kg of water in the pond, there is 1 mole of concrete in contact with water.

In [6]:
Por = aqequil.Reactant(reactant_name="portlandite",
                       amount_remaining=0.1, # moles of mineral to react per kg H2O
                       f_rate_law='Relative rate equation', # forward rate law
                       f_eq1=1, # dXi(n)/dXi (mol/mol), the mole fraction of this mineral in the rock
                       b_rate_law="Partial equilibrium", # backward rate law
                       )

# combine the mineral reactants in a list to prepare the reaction
r = aqequil.Prepare_Reaction(reactants=[Por])

In [7]:
speciation = aqequil.react(speciation, r)

Using wrm_data_latest.csv to react Henry.Clay.Sterling.Mine


In [8]:
speciation.lookup('name')

Column(s) not found: ['name']


Sample
Henry Clay Sterling Mine


In [9]:
# get mass transfer data for a sample
m = speciation.mt("Henry Clay Sterling Mine")

## Activity Answers

First, plot product mineral volumes (not log scale) and make sure reactant minerals are shown.

Isolate the portlandite trend.

In [10]:
m.plot_product_minerals(show_reactant_minerals=True, log_y=False, y_type='volume')

The starting volume of portlandite is 3.31 cm3 and it ends at 2.70 cm3.

Calculate the percentage of portlandite volume that has dissolved in the AMD:

In [11]:
100 - (2.70/3.31)*100

18.42900302114802

18% > 10% budget choice threshold. Allocate \\$5M.